# Notebook for looking at recorded phone usage

In [ ]:
import os
import sys
import glob
import pandas as pd

from datetime import timedelta

sys.path.insert(0, "../")
from db import HindsightDB
import utils

In [ ]:
db = HindsightDB()

In [ ]:
frames_df = db.get_frames()
frames_df = utils.add_datetimes(frames_df)
frames_df = frames_df.sort_values(by='datetime_local')

In [ ]:
set(frames_df['application'])

In [ ]:
frames_df = frames_df.loc[frames_df['application'] != 'com-android-systemui'] # Lines up better with phone

In [ ]:
frames_df['time_difference'] = frames_df['datetime_local'].diff()

In [ ]:
new_usage_threshold = timedelta(seconds=120)
new_usage_start = (frames_df['time_difference'] > new_usage_threshold)
frames_df['usage_id'] = new_usage_start.cumsum()

In [ ]:
frames_df['datetime_to_day'] = frames_df['datetime_local'].dt.floor('D')

In [ ]:
frames_df.groupby(['datetime_to_day']).usage_id.nunique()

In [ ]:
usage_durations = frames_df.groupby('usage_id')['datetime_local'].agg([min, max])
usage_durations['duration'] = usage_durations['max'] - usage_durations['min']
usage_durations['day'] = usage_durations['min'].dt.floor('D')

In [ ]:
usage_durations.groupby(['day']).duration.sum()

In [ ]:
usage_durations.duration.sum()